# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from config import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("Output_Data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,atafu village,-8.5421,-172.5159,28.88,72,55,8.60,TK,1690254896
1,1,longyearbyen,78.2186,15.6401,9.91,76,75,4.12,SJ,1690254872
2,2,san patricio,28.0170,-97.5169,30.10,69,49,5.06,US,1690254885
3,3,kayangel,8.0823,134.7173,28.58,74,99,4.16,PW,1690254977
4,4,ebaye,8.7748,167.7392,32.01,62,40,5.66,MH,1690254854


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True, 
                                      tiles = "OSM", 
                                      frame_width = 800, 
                                      frame_height = 600,
                                      size = "Humidity", 
                                      scale = 1, 
                                      color = "City"
                                     )

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[(city_data_df["Max Temp"] < 26) &
                            (city_data_df["Max Temp"] > 21) &
                            (city_data_df["Cloudiness"] < 10) &
                            (city_data_df["Wind Speed"] < 10) &
                            (city_data_df["Humidity"] > 60)]

# Drop any rows with null values
ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
44,44,pacific grove,36.6177,-121.9166,21.30,74,0,5.14,US,1690254978
98,98,sao joao da barra,-21.6403,-41.0511,21.03,87,3,3.82,BR,1690254657
113,113,hamilton,39.1834,-84.5333,22.80,88,0,2.06,US,1690254497
129,129,vilyuchinsk,52.9306,158.4028,25.15,63,1,3.82,RU,1690254904
145,145,bethel,41.3712,-73.4140,24.20,81,0,1.54,US,1690254872
192,192,martinsburg,39.4562,-77.9639,24.79,92,0,0.00,US,1690255034
297,297,altamira,-3.2033,-52.2064,22.65,61,1,0.94,BR,1690255045
402,402,brookings,44.3114,-96.7984,24.00,76,0,3.09,US,1690254882
411,411,marshfield,44.6689,-90.1718,21.49,78,0,0.00,US,1690255059
423,423,mancio lima,-7.6142,-72.8958,24.04,94,0,0.51,BR,1690255061


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
44,pacific grove,US,36.6177,-121.9166,74,
98,sao joao da barra,BR,-21.6403,-41.0511,87,
113,hamilton,US,39.1834,-84.5333,88,
129,vilyuchinsk,RU,52.9306,158.4028,63,
145,bethel,US,41.3712,-73.4140,81,
192,martinsburg,US,39.4562,-77.9639,92,
297,altamira,BR,-3.2033,-52.2064,61,
402,brookings,US,44.3114,-96.7984,76,
411,marshfield,US,44.6689,-90.1718,78,
423,mancio lima,BR,-7.6142,-72.8958,94,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel", 
          "apiKey": geoapify_key
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
pacific grove - nearest hotel: Pacific Grove Inn
sao joao da barra - nearest hotel: Pousada Mediterrâneo
hamilton - nearest hotel: North Vista Manor
vilyuchinsk - nearest hotel: Победа
bethel - nearest hotel: Hampton Inn Danbury
martinsburg - nearest hotel: Historic Boydville
altamira - nearest hotel: Hotel Vale do Xingu
brookings - nearest hotel: Quality Inn
marshfield - nearest hotel: Holiday Inn Conference Center Marshfield
mancio lima - nearest hotel: No hotel found
kill devil hills - nearest hotel: Mariner Days Inn & Suites
melfort - nearest hotel: No hotel found
muana - nearest hotel: No hotel found
westport - nearest hotel: Norwalk Inn and Conference Center
morris - nearest hotel: Candlewood
salinas - nearest hotel: The Traveler's Hotel
taroudant - nearest hotel: Hotel Tiouit


,City,Country,Lat,Lng,Humidity,Hotel Name
44,pacific grove,US,36.6177,-121.9166,74,Pacific Grove Inn
98,sao joao da barra,BR,-21.6403,-41.0511,87,Pousada Mediterrâneo
113,hamilton,US,39.1834,-84.5333,88,North Vista Manor
129,vilyuchinsk,RU,52.9306,158.4028,63,Победа
145,bethel,US,41.3712,-73.4140,81,Hampton Inn Danbury
192,martinsburg,US,39.4562,-77.9639,92,Historic Boydville
297,altamira,BR,-3.2033,-52.2064,61,Hotel Vale do Xingu
402,brookings,US,44.3114,-96.7984,76,Quality Inn
411,marshfield,US,44.6689,-90.1718,78,Holiday Inn Conference Center Marshfield
423,mancio lima,BR,-7.6142,-72.8958,94,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True, 
                                      tiles = "OSM", 
                                      frame_width = 850, 
                                      frame_height = 600,
                                      size = "Humidity", 
                                      scale = 1, 
                                      color = "City"
                                     )

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)